# Processamento de dados com Spark nas eleições de 2022

## Passos iniciais e Lazy Evaluation

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkEleicoes2022').getOrCreate()

/home/joaopedro/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Carregando arquivo de bens declarados dos candidatos

In [8]:
bens_df = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv', 
        header=True, 
        inferSchema=True,
        sep=';',
        encoding='latin-1'
    )
)

Tentativa vã de mostrar o conteúdo do arquivo

In [9]:
print(bens_df)

DataFrame[DT_GERACAO: string, HH_GERACAO: string, ANO_ELEICAO: string, CD_TIPO_ELEICAO: string, NM_TIPO_ELEICAO: string, CD_ELEICAO: int, DS_ELEICAO: string, DT_ELEICAO: string, SG_UF: string, SG_UE: string, NM_UE: string, SQ_CANDIDATO: bigint, NR_ORDEM_CANDIDATO: int, CD_TIPO_BEM_CANDIDATO: int, DS_TIPO_BEM_CANDIDATO: string, DS_BEM_CANDIDATO: string, VR_BEM_CANDIDATO: string, DT_ULTIMA_ATUALIZACAO: string, HH_ULTIMA_ATUALIZACAO: string]


Oops! O Spark não carregou o arquivo, apenas criou um objeto que representa o arquivo. Isso é chamado de **lazy evaluation**. O Spark só vai carregar o arquivo quando for necessário.

Mostrando as 10 primeiras linhas do conjunto de dados do spak

In [16]:
(
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv', 
        header=True, 
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
    # Até essa linha o Spark não faz nada, apenas monta o pipeline
    # A partir daqui o Spark começa a processar os dados
    # pois o método .show() é um  lazy evaluation
    .show(10, True)
)

+----------+----------+-----------+---------------+-----------------+----------+--------------------+----------+-----+-----+-------------------+------------+------------------+---------------------+---------------------+--------------------+----------------+---------------------+---------------------+
|DT_GERACAO|HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|CD_ELEICAO|          DS_ELEICAO|DT_ELEICAO|SG_UF|SG_UE|              NM_UE|SQ_CANDIDATO|NR_ORDEM_CANDIDATO|CD_TIPO_BEM_CANDIDATO|DS_TIPO_BEM_CANDIDATO|    DS_BEM_CANDIDATO|VR_BEM_CANDIDATO|DT_ULTIMA_ATUALIZACAO|HH_ULTIMA_ATUALIZACAO|
+----------+----------+-----------+---------------+-----------------+----------+--------------------+----------+-----+-----+-------------------+------------+------------------+---------------------+---------------------+--------------------+----------------+---------------------+---------------------+
|25/09/2022|  18:35:28|       2022|              2|Eleição Ordinária|       546|Eleições Ge

Agora sim! O Spark carregou o arquivo e mostrou as 10 primeiras linhas.

## Primeiras queries e planos de execução

In [17]:
from pyspark.sql import functions as F

In [20]:
# Apontando para o diretório onde estão os arquivos
bens_df = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv', 
        header=True, 
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
)

# Realizando a query

(
    bens_df
    .select(
        [
            'SQ_CANDIDATO', 
            'DS_TIPO_BEM_CANDIDATO', 
            'VR_BEM_CANDIDATO'
        ]
    )
    .withColumn(
        # substituindo , por . para que o Spark entenda que é um float
        # e convertendo para float
        
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('float')
    )
    .show(10, True)
)

+------------+---------------------+----------------+
|SQ_CANDIDATO|DS_TIPO_BEM_CANDIDATO|VR_BEM_CANDIDATO|
+------------+---------------------+----------------+
|110001608768| Veículo automotor...|         40000.0|
|200001608811| Fundos: Ações, Mú...|        50164.28|
|240001614377|              Terreno|        186200.0|
|240001614377| Caderneta de poup...|         2367.39|
|210001647159| Depósito bancário...|        22713.17|
|180001610759| Outras aplicações...|       222349.52|
|180001610760|                 Casa|       1300000.0|
|250001611430|                 Casa|        511700.0|
|250001611430| Veículo automotor...|         80000.0|
| 80001651647| Outras aplicações...|        68598.68|
+------------+---------------------+----------------+
only showing top 10 rows



Visualizando o plano de execução da query com filtro

In [29]:
query_bens_df = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv', 
        header=True, 
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
    .select(
        [
            'SQ_CANDIDATO', 
            'DS_TIPO_BEM_CANDIDATO', 
            'VR_BEM_CANDIDATO'
        ]
    )
    .withColumn(
        # substituindo , por . para que o Spark entenda que é um float
        # e convertendo para float
        
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('float')
    )
    
)

# filtrar os bens com valor maior que 10 mil
query_bens_df = query_bens_df.filter('VR_BEM_CANDIDATO > 10000')
query_bens_df.explain(mode='formatted')

== Physical Plan ==
* Project (3)
+- * Filter (2)
   +- Scan csv  (1)


(1) Scan csv 
Output [3]: [SQ_CANDIDATO#1256L, DS_TIPO_BEM_CANDIDATO#1259, VR_BEM_CANDIDATO#1261]
Batched: false
Location: InMemoryFileIndex [file:/home/joaopedro/Desktop/projects/analise_bens_deputados_fed/bem_candidato_2022/bem_candidato_2022_BRASIL.csv]
PushedFilters: [IsNotNull(VR_BEM_CANDIDATO)]
ReadSchema: struct<SQ_CANDIDATO:bigint,DS_TIPO_BEM_CANDIDATO:string,VR_BEM_CANDIDATO:string>

(2) Filter [codegen id : 1]
Input [3]: [SQ_CANDIDATO#1256L, DS_TIPO_BEM_CANDIDATO#1259, VR_BEM_CANDIDATO#1261]
Condition : (isnotnull(VR_BEM_CANDIDATO#1261) AND (cast(regexp_replace(VR_BEM_CANDIDATO#1261, ,, ., 1) as float) > 10000.0))

(3) Project [codegen id : 1]
Output [3]: [SQ_CANDIDATO#1256L, DS_TIPO_BEM_CANDIDATO#1259, cast(regexp_replace(VR_BEM_CANDIDATO#1261, ,, ., 1) as float) AS VR_BEM_CANDIDATO#1286]
Input [3]: [SQ_CANDIDATO#1256L, DS_TIPO_BEM_CANDIDATO#1259, VR_BEM_CANDIDATO#1261]




## Agregações e partições

Vamos calcular o total de bens declarados por todos os candidatos a cargos de deputado federal

In [37]:
# Vamos calcular o total de bens declarados por todos os candidatos
agg_total_bens = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv', 
        header=True, 
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
    .select(
        [
            'SQ_CANDIDATO',
            'VR_BEM_CANDIDATO'
        ]
    )
    .withColumn(
        # substituindo , por . para que o Spark entenda que é um float
        # e convertendo para float
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('float')
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('TOTAL_BENS')
    )
)

In [38]:
agg_total_bens.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[sum(VR_BEM_CANDIDATO#1769)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#838]
      +- HashAggregate(keys=[], functions=[partial_sum(VR_BEM_CANDIDATO#1769)])
         +- Project [cast(regexp_replace(VR_BEM_CANDIDATO#1745, ,, ., 1) as float) AS VR_BEM_CANDIDATO#1769]
            +- FileScan csv [VR_BEM_CANDIDATO#1745] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/joaopedro/Desktop/projects/analise_bens_deputados_fed/bem_c..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<VR_BEM_CANDIDATO:string>




In [39]:
agg_total_bens.show(truncate=False)

+---------------------+
|TOTAL_BENS           |
+---------------------+
|2.3072723030748222E10|
+---------------------+



## Joins e Spark SQL

In [40]:
# 'Carregando' o arquivo de bens
bens_df = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv',
        header=True,
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
    .withColumn(
        # substituindo , por . para que o Spark entenda que é um float
        # e convertendo para float
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('float')
    )
)


# Carregando o arquivo de candidatos
candidatos_df = (
    spark
    .read
    .csv(
        './consulta_cand_2022/consulta_cand_2022_BRASIL.csv',
        header=True,
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
)

Realizando o Join na chave SQ_CANDIDATO

In [44]:
# Join entre os dois DataFrames
bens_candidatos_df = bens_df.join(
    candidatos_df,
    'SQ_CANDIDATO',
    how = 'left'
)

# Visualizando as 10 primeiras linhas
bens_candidatos_df.select(
    [
        'SQ_CANDIDATO',
        'NM_CANDIDATO',
        'DS_TIPO_BEM_CANDIDATO',
        'SG_PARTIDO',
        'VR_BEM_CANDIDATO'
    ]
).show(10, True)

+------------+--------------------+---------------------+----------+----------------+
|SQ_CANDIDATO|        NM_CANDIDATO|DS_TIPO_BEM_CANDIDATO|SG_PARTIDO|VR_BEM_CANDIDATO|
+------------+--------------------+---------------------+----------+----------------+
|110001608768|   VANDERLEY DA GUIA| Veículo automotor...|      PSOL|         40000.0|
|200001608811|WALTER PEREIRA ALVES| Fundos: Ações, Mú...|        PT|        50164.28|
|240001614377|SILVIO ALEXANDRE ...|              Terreno|       PSD|        186200.0|
|240001614377|SILVIO ALEXANDRE ...| Caderneta de poup...|       PSD|         2367.39|
|210001647159| JOÃO ERVINO FISCHER| Depósito bancário...|        PP|        22713.17|
|180001610759|FLAVIO RODRIGUES ...| Outras aplicações...|        PT|       222349.52|
|180001610760|JADYEL SILVA ALENCAR|                 Casa|        PV|       1300000.0|
|250001611430|RENATO PUPO DE PAULA|                 Casa|      PSDB|        511700.0|
|250001611430|RENATO PUPO DE PAULA| Veículo automotor.

Realizando a mesma query com Spark SQL

In [47]:
# Primeiramente, as tabelas são cadastradas na interface SQL
# para que possam ser referenciadas no Spark SQL
bens_df.createOrReplaceTempView('bens')
candidatos_df.createOrReplaceTempView('candidatos')

# Realizando a query
# pura e simplesmente usando SQL
bens_candidatos_df = spark.sql(
    """
    SELECT
        bens.SQ_CANDIDATO,
        candidatos.NM_CANDIDATO,
        bens.DS_TIPO_BEM_CANDIDATO,
        candidatos.SG_PARTIDO,
        bens.VR_BEM_CANDIDATO
    FROM
        bens
    LEFT JOIN
        candidatos
    ON
        bens.SQ_CANDIDATO = candidatos.SQ_CANDIDATO
    """
)

bens_candidatos_df.show(10, True)

+------------+--------------------+---------------------+----------+----------------+
|SQ_CANDIDATO|        NM_CANDIDATO|DS_TIPO_BEM_CANDIDATO|SG_PARTIDO|VR_BEM_CANDIDATO|
+------------+--------------------+---------------------+----------+----------------+
|110001608768|   VANDERLEY DA GUIA| Veículo automotor...|      PSOL|         40000.0|
|200001608811|WALTER PEREIRA ALVES| Fundos: Ações, Mú...|        PT|        50164.28|
|240001614377|SILVIO ALEXANDRE ...|              Terreno|       PSD|        186200.0|
|240001614377|SILVIO ALEXANDRE ...| Caderneta de poup...|       PSD|         2367.39|
|210001647159| JOÃO ERVINO FISCHER| Depósito bancário...|        PP|        22713.17|
|180001610759|FLAVIO RODRIGUES ...| Outras aplicações...|        PT|       222349.52|
|180001610760|JADYEL SILVA ALENCAR|                 Casa|        PV|       1300000.0|
|250001611430|RENATO PUPO DE PAULA|                 Casa|      PSDB|        511700.0|
|250001611430|RENATO PUPO DE PAULA| Veículo automotor.